In [1]:
!apt-get install git -y



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
!git clone https://github.com/tdarthub/finetuning-.git


Cloning into 'finetuning-'...


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.2 MB/s eta 0:00:00


In [ ]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8


NotImplementedError: ignored

In [ ]:
!pip install transformers

NotImplementedError: ignored

In [ ]:
from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import tqdm
import torch

class ChatData(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> " + i + " <doctor>: " + self.X[idx + 1] + " <endofstring>"
            except:
                break

        self.X_encoded = tokenizer(self.X, max_length=150, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

def train(chatData, model, optimizer):
    epochs = 20
    model.train()

    for epoch in tqdm.trange(epochs, desc="Epoch"):
        epoch_loss = 0.0

        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)

            optimizer.zero_grad()

            outputs = model(X, attention_mask=a, labels=X)
            loss = outputs.loss
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(chatData)}")

        # Save the model checkpoint after each epoch
        torch.save(model.state_dict(), f"model_state_epoch_{epoch + 1}.pt")

def infer(inp):
    inp = "<startofstring> " + inp + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp, max_length=80)
    output = tokenizer.decode(output[0])

    return output

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<doctor>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

chatData = ChatData("/content/drive/MyDrive/DATASET_self.json", tokenizer)
chatData = DataLoader(chatData, batch_size=32)

optimizer = AdamW(model.parameters(), lr=1e-4)

print("Training...")
train(chatData, model, optimizer)

print("Infer from the model:")
while True:
    inp = input()
    print(infer(inp))


Exception ignored in: <function _xla_gc_callback at 0x7f1874e26170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

text = "No problem at all. Meditation is a way to calm your mind and reduce stress. Find a quiet place where you can sit comfortably. Close your eyes, take a deep breath, and try to focus on your breath. If thoughts come up, just notice them and let them go without getting too caught up in them. Bring your attention back to your breath. Start with just a few minutes each day and gradually increase the time. Meditation can bring a sense of peace and help you manage your irritability. Would you be open to giving it a try?"

encoded_text = tokenizer.encode(text)
token_count = len(encoded_text)

print("Token count:", token_count)


Token count: 111


In [ ]:
#bigger dataset

from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import tqdm
import torch

class ChatData(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> " + i + " <doctor>: " + self.X[idx + 1] + " <endofstring>"
            except:
                break

        self.X_encoded = tokenizer(self.X, max_length=150, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

def train(chatData, model, optimizer):
    epochs = 40
    model.train()

    for epoch in tqdm.trange(epochs, desc="Epoch"):
        epoch_loss = 0.0

        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)

            optimizer.zero_grad()

            outputs = model(X, attention_mask=a, labels=X)
            loss = outputs.loss
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(chatData)}")

        # Save the model checkpoint after each epoch
        torch.save(model.state_dict(), f"model_state_epoch_{epoch + 1}.pt")

def infer(inp):
    inp = "<startofstring> " + inp + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp, max_length=80)
    output = tokenizer.decode(output[0])

    return output

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<doctor>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

chatData = ChatData("drive/MyDrive/updated_dataset.json", tokenizer)
chatData = DataLoader(chatData, batch_size=32)

optimizer = AdamW(model.parameters(), lr=1e-4)

print("Training...")
train(chatData, model, optimizer)

print("Infer from the model:")
while True:
    inp = input()
    print(infer(inp))


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training...


Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
print("Infer from the model:")
while True:
    inp = input()
    print(infer(inp))

'/content'

In [ ]:
from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import tqdm
import torch
import os

class ChatData(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> " + i + " <doctor>: " + self.X[idx + 1] + " <endofstring>"
            except:
                break

        self.X_encoded = tokenizer(self.X, max_length=150, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

def train(chatData, model, optimizer):
    epochs = 50
    model.train()

    for epoch in tqdm.trange(epochs, desc="Epoch"):
        epoch_loss = 0.0

        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)

            optimizer.zero_grad()

            outputs = model(X, attention_mask=a, labels=X)
            loss = outputs.loss
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(chatData)}")

        # Save the model checkpoint after every 10 epochs
        if (epoch + 1) % 10 == 0:
            model_dir = "models"
            os.makedirs(model_dir, exist_ok=True)
            checkpoint_path = os.path.join(model_dir, f"model_state_epoch_{epoch + 1}.pt")
            torch.save(model.state_dict(), checkpoint_path)

            # Clear GPU memory
            torch.cuda.empty_cache()

def infer(inp):
    inp = "<startofstring> " + inp + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp, max_length=80)
    output = tokenizer.decode(output[0])

    return output

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<doctor>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

chatData = ChatData("drive/MyDrive/updated_dataset.json", tokenizer)
chatData = DataLoader(chatData, batch_size=32)

optimizer = AdamW(model.parameters(), lr=1e-4)

print("Training...")
train(chatData, model, optimizer)




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training...


Epoch:   2%|▏         | 1/50 [04:01<3:17:30, 241.84s/it]

Epoch 1 Loss: 2.5612741946850135


Epoch:   4%|▍         | 2/50 [08:06<3:14:55, 243.65s/it]

...

Epoch 34 Loss: 0.15701072005069125


Epoch:  70%|███████   | 35/50 [2:22:47<1:01:04, 244.31s/it]

Epoch 35 Loss: 0.15082654470334883


Epoch:  72%|███████▏  | 36/50 [2:26:50<56:58, 244.16s/it]  

Epoch 36 Loss: 0.1455810593198175


In [ ]:
print("Infer from the model:")
while True:
    inp = input()
    print(infer(inp))

In [ ]:
#training continue
from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import tqdm
import torch
import os

class ChatData(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> " + i + " <doctor>: " + self.X[idx + 1] + " <endofstring>"
            except:
                break

        self.X_encoded = tokenizer(self.X, max_length=150, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

def train(chatData, model, optimizer):
    epochs = 50
    model.train()

    for epoch in tqdm.trange(epochs, desc="Epoch"):
        epoch_loss = 0.0

        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)

            optimizer.zero_grad()

            outputs = model(X, attention_mask=a, labels=X)
            loss = outputs.loss
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(chatData)}")

        # Save the model checkpoint after every 10 epochs
        if (epoch + 1) % 10 == 0:
            model_dir = "models"
            os.makedirs(model_dir, exist_ok=True)
            checkpoint_path = os.path.join(model_dir, f"model_state_epoch_{epoch + 1}.pt")
            torch.save(model.state_dict(), checkpoint_path)

            # Clear GPU memory
            torch.cuda.empty_cache()

def infer(inp):
    inp = "<startofstring> " + inp + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp, max_length=80)
    output = tokenizer.decode(output[0])

    return output

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<doctor>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

chatData = ChatData("/content/drive/MyDrive/updated_dataset.json", tokenizer)
chatData = DataLoader(chatData, batch_size=32)

optimizer = AdamW(model.parameters(), lr=1e-4)

print("Training...")

# Load the model checkpoint for resuming from the 30th epoch
checkpoint_path = "/content/drive/MyDrive/updated/model_state_epoch_30.pt"
model.load_state_dict(torch.load(checkpoint_path))

train(chatData, model, optimizer)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training...


Epoch:   2%|▏         | 1/50 [03:59<3:15:46, 239.73s/it]

Epoch 1 Loss: 0.17874544891326324


Epoch:   4%|▍         | 2/50 [08:01<3:12:41, 240.86s/it]

...

Epoch:  96%|█████████▌| 48/50 [3:13:48<08:05, 242.71s/it]

Epoch 48 Loss: 0.07515272975939771


Epoch:  98%|█████████▊| 49/50 [3:17:50<04:02, 242.69s/it]

Epoch 49 Loss: 0.07519696140661836
Epoch 50 Loss: 0.07532900616364635


Epoch: 100%|██████████| 50/50 [3:21:54<00:00, 242.30s/it]


In [ ]:
def infer(question, context):
    inp = "<startofstring> " + context + question + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp.to(device), max_length=1024)
    output = tokenizer.decode(output[0])

    return output



In [ ]:
print("Infer from the model:")
context = ""
while True:
    question = input("Your question: ")
    if question.lower() == "quit":
        break
    response = infer(question, context)
    print("Doctor:", response)
    context += "You: " + question + " Doctor: " + response + " "


Infer from the model:
Your question: hello


NameError: ignored

In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
!python /content/drive/MyDrive/updated/model_state_epoch_30.pt


/usr/bin/python3: can't find '__main__' module in '/content/drive/MyDrive/updated/model_state_epoch_30.pt'


In [ ]:
!pip install torch


In [ ]:
model = torch.load(model_path, map_location=torch.device('cpu'))


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model checkpoint
model_path = "/content/drive/MyDrive/updated/model_state_epoch_50.pt"
model = GPT2LMHeadModel.from_pretrained("gpt2")


# Set up tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_response(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Interaction loop
print("Model loaded. Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = generate_response(user_input)
    print(f"Model: {response}")


You: hi


KeyboardInterrupt: ignored

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def infer(model, tokenizer, inp, max_length=80):
    inp = "<startofstring> " + inp + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp, max_length=max_length, num_return_sequences=1)
    output = tokenizer.decode(output[0], skip_special_tokens=True)

    return output

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<doctor>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

# Load the trained model checkpoint
checkpoint_path = "/content/drive/MyDrive/updated/model_state_epoch_50.pt"
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    response = infer(model, tokenizer, user_input)
    print(f"Bot: {response}")


You: hello


KeyboardInterrupt: ignored

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import requests
from bs4 import BeautifulSoup

def infer(model, tokenizer, inp, max_length=80):
    inp = "<startofstring> " + inp + " <doctor>: "
    inp = tokenizer.encode(inp, return_tensors="pt")
    inp = inp.to(device)

    output = model.generate(inp, max_length=max_length, num_return_sequences=1)
    output = tokenizer.decode(output[0], skip_special_tokens=True)

    return output

def google_search(query):
    response = requests.get(f"https://www.google.com/search?q={query}")
    soup = BeautifulSoup(response.text, "html.parser")

    results = soup.find_all("div", class_="g")
    if results:
        return results[0].text
    else:
        return None

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startofstring>",
                              "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<doctor>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

# Load the trained model checkpoint
checkpoint_path = "/content/drive/MyDrive/updated/model_state_epoch_50.pt"
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    response = infer(model, tokenizer, user_input)
    print(f"Bot: {response}")

    if response:
        print("Google Search:")
        print(google_search(response))
    else:
        print("Google Search: No results found.")


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50261. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


You: hello


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: hello <doctor>: Hi! What can I do for you today?
Google Search:
None
You: tell me what i do


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Bot: last night i saw a very orrible dreem <doctor>: I'm sorry to hear that. It takes a lot of courage to seek help. Can you tell me more about your substance use?
Google Search:
None
You: no i did not use any substance i said i watced a very horror dreem tomorrow night 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: no i did not use any substance i said i watced a very horror dreem tomorrow night <doctor>: I'm glad it helped you feel more relaxed. In addition to sleep and diet, I'd like to suggest incorporating some relaxation techniques into your child's routine. One effective technique is meditation. It's a simple practice of focusing on their breath and being present in the moment. It can help
Google Search:
None
You: but i dont want to meditate 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: but i dont want to meditate <doctor>: I understand that it's hard, but it's important that you take this seriously. We need to come up with a safety plan to keep you from harming yourself. Would you be willing to work on that with me?
Google Search:
None


In [ ]:
import torch
import warnings
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import requests
from bs4 import BeautifulSoup

def infer(model, tokenizer, inp, max_length=80):
    input_ids = tokenizer.encode(inp, add_special_tokens=True)
    input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    output = tokenizer.decode(output[0], skip_special_tokens=True)

    return output

def google_search(query):
    response = requests.get(f"https://www.google.com/search?q={query}")
    soup = BeautifulSoup(response.text, "html.parser")

    results = soup.find_all("div", class_="g")
    if results:
        return results[0].text
    else:
        return None

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(device)

# Load the trained model checkpoint
checkpoint_path = "/content/drive/MyDrive/updated/model_state_epoch_50.pt"
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        response = infer(model, tokenizer, user_input)
    print(f"Bot: {response}")

    if response:
        print("Google Search:")
        print(google_search(response))
    else:
        print("Google Search: No results found.")


RuntimeError: ignored